In [104]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [88]:
# Load filtered training data
tr_Acc = np.load("train_Accelerometer_OpenDoor_RubHands.npy")
tr_Grav = np.load("train_Gravity_OpenDoor_RubHands.npy")
tr_labels = np.load("train_labels_OpenDoor_RubHands.npy")

# Load filtered testing data
ts_Acc = np.load("test_Accelerometer_OpenDoor_RubHands.npy")
ts_Grav = np.load("test_Gravity_OpenDoor_RubHands.npy")
ts_labels = np.load("test_labels_OpenDoor_RubHands.npy")

 # converting labels to binary
tr_labels = np.where(tr_labels == 20, 0, 1)
ts_labels = np.where(ts_labels == 20, 0, 1)


In [89]:
# Check the shapes of the loaded data
print("Training Accelerometer shape:", tr_Acc.shape)
print("Training Gravity shape:", tr_Grav.shape)
print("Training Labels shape:", tr_labels.shape)

print("Testing Accelerometer shape:", ts_Acc.shape)
print("Testing Gravity shape:", ts_Grav.shape)
print("Testing Labels shape:", ts_labels.shape)

Training Accelerometer shape: (87, 800, 3)
Training Gravity shape: (87, 800, 3)
Training Labels shape: (87,)
Testing Accelerometer shape: (90, 800, 3)
Testing Gravity shape: (90, 800, 3)
Testing Labels shape: (90,)


In [90]:
# Set number of training and testing examples
train_num_examples = 87
test_num_examples = 90

num_sensors = 2
num_features_per_axis = 8
num_axes = 3

train_features = np.zeros((train_num_examples, num_sensors * num_features_per_axis, num_axes))
test_features = np.zeros((test_num_examples, num_sensors * num_features_per_axis, num_axes))

In [91]:
# Generation of features for training dataset
# Accelerometer features
train_features[:, 0, :] = np.mean(tr_Acc, axis=1)
train_features[:, 1, :] = np.max(tr_Acc, axis=1)
train_features[:, 2, :] = np.min(tr_Acc, axis=1)
train_features[:, 3, :] = np.std(tr_Acc, axis=1)
train_features[:, 4, :] = np.sum(np.diff(np.sign(tr_Acc), axis=1) != 0, axis=1)
train_features[:, 5, :] = np.percentile(tr_Acc, 20, axis=1)
train_features[:, 6, :] = np.percentile(tr_Acc, 50, axis=1)
train_features[:, 7, :] = np.percentile(tr_Acc, 80, axis=1)

# Gravity features
train_features[:, 8, :] = np.mean(tr_Grav, axis=1)
train_features[:, 9, :] = np.max(tr_Grav, axis=1)
train_features[:, 10, :] = np.min(tr_Grav, axis=1)
train_features[:, 11, :] = np.std(tr_Grav, axis=1)
train_features[:, 12, :] = np.sum(np.diff(np.sign(tr_Grav), axis=1) != 0, axis=1)
train_features[:, 13, :] = np.percentile(tr_Grav, 20, axis=1)
train_features[:, 14, :] = np.percentile(tr_Grav, 50, axis=1)
train_features[:, 15, :] = np.percentile(tr_Grav, 80, axis=1)

# Generation of features for testing dataset
# Accelerometer features
test_features[:, 0, :] = np.mean(ts_Acc, axis=1)
test_features[:, 1, :] = np.max(ts_Acc, axis=1)
test_features[:, 2, :] = np.min(ts_Acc, axis=1)
test_features[:, 3, :] = np.std(ts_Acc, axis=1)
test_features[:, 4, :] = np.sum(np.diff(np.sign(ts_Acc), axis=1) != 0, axis=1)
test_features[:, 5, :] = np.percentile(ts_Acc, 20, axis=1)
test_features[:, 6, :] = np.percentile(ts_Acc, 50, axis=1)
test_features[:, 7, :] = np.percentile(ts_Acc, 80, axis=1)

# Gravity features
test_features[:, 8, :] = np.mean(ts_Grav, axis=1)
test_features[:, 9, :] = np.max(ts_Grav, axis=1)
test_features[:, 10, :] = np.min(ts_Grav, axis=1)
test_features[:, 11, :] = np.std(ts_Grav, axis=1)
test_features[:, 12, :] = np.sum(np.diff(np.sign(ts_Grav), axis=1) != 0, axis=1)
test_features[:, 13, :] = np.percentile(ts_Grav, 20, axis=1)
test_features[:, 14, :] = np.percentile(ts_Grav, 50, axis=1)
test_features[:, 15, :] = np.percentile(ts_Grav, 80, axis=1)

print("Train features shape:", train_features.shape)
print("Test features shape:", test_features.shape)


Train features shape: (87, 16, 3)
Test features shape: (90, 16, 3)


In [92]:
train_features_reshaped = np.reshape(train_features, (train_features.shape[0], train_features.shape[1] * train_features.shape[2]))
test_features_reshaped = np.reshape(test_features, (test_features.shape[0], test_features.shape[1] * test_features.shape[2]))

print("Train features reshaped shape:", train_features_reshaped.shape)
print("Test features reshaped shape:", test_features_reshaped.shape)

Train features reshaped shape: (87, 48)
Test features reshaped shape: (90, 48)


In [139]:
train_features_tensor = torch.from_numpy(train_features_reshaped).float()
test_features_tensor = torch.from_numpy(test_features_reshaped).float()
train_labels_tensor = torch.from_numpy(tr_labels).float().view(-1, 1)
test_labels_tensor = torch.from_numpy(ts_labels).float().view(-1, 1)

batch_size = 50
learning_rate = 0.01
epochs = 100

#OPEN_DOOR == 0
#RUB_HANDS == 1

#Create DataLoader for batch processing
train_dataset = TensorDataset(train_features_tensor, train_labels_tensor)
test_dataset = TensorDataset(test_features_tensor, test_labels_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [142]:
# Neural Network Model

class BinaryClassificationModel(nn.Module):
    def __init__(self, input_size):
        super(BinaryClassificationModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.sigmoid(self.fc4(x))
        return x

input_size = train_features_reshaped.shape[1]  # 48 features
model = BinaryClassificationModel(input_size)

criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr= learning_rate, momentum=0.9)

# Train the model
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    if (epoch + 1) % 100 == 0:
      print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Evaluate the model on the test data
model.eval()
correct = 0
total = 0
predict_list = []
test_list = []
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predicted = (outputs > 0.5).float()  # Convert sigmoid output to binary (0 or 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        predict_list.append(predicted)
        test_list.append(labels)

predict_list = torch.cat(predict_list)
test_list = torch.cat(test_list)

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

print("Classification Report:")
print(classification_report(test_list,predict_list))
print("Confusion Matrix:")
print(confusion_matrix(test_list,predict_list))


Epoch [100/100], Loss: 0.0173
Test Accuracy: 94.44%
Classification Report:
              precision    recall  f1-score   support

         0.0       0.91      1.00      0.95        48
         1.0       1.00      0.88      0.94        42

    accuracy                           0.94        90
   macro avg       0.95      0.94      0.94        90
weighted avg       0.95      0.94      0.94        90

Confusion Matrix:
[[48  0]
 [ 5 37]]


In [138]:
# Logistic Regression Model

log_reg_classifier = LogisticRegression(solver='liblinear')
log_reg_classifier.fit(train_features_reshaped, tr_labels)

#Predict on the test data
estimatedLabels = log_reg_classifier.predict(test_features_reshaped)

print("Classification Report:")
print(classification_report(test_labels_tensor, estimatedLabels))
print("Confusion Matrix:")
print(confusion_matrix(test_labels_tensor, estimatedLabels))

Classification Report:
              precision    recall  f1-score   support

         0.0       0.92      1.00      0.96        48
         1.0       1.00      0.90      0.95        42

    accuracy                           0.96        90
   macro avg       0.96      0.95      0.95        90
weighted avg       0.96      0.96      0.96        90

Confusion Matrix:
[[48  0]
 [ 4 38]]
